# WSI - laboratorium 7

Zadanie: Modele bayesowskie

Autor: Jakub Mazurkiewicz (300226)

## Implementacja naiwnego klasyfikatora Bayesa

In [31]:
import numpy as np

class NaiveBayes:
    def __init__(self, x_train, y_train):
        sample_count, attrib_count = x_train.shape
        self.classes = np.unique(y_train)
        self.class_count = len(self.classes)

        self.mean = np.zeros((self.class_count, attrib_count))
        self.stdev = np.zeros((self.class_count, attrib_count))
        self.prob = np.zeros([self.class_count])

        for i, c in enumerate(self.classes):
            x_of_c = x_train[y_train == c]
            self.mean[i] = x_of_c.mean(axis=0)
            self.stdev[i] = x_of_c.std(axis=0)
            self.prob[i] = x_of_c.shape[0] / sample_count

    def classify(self, x):
        return [self._classify_one(one) for one in x]

    def _classify_one(self, x):
        search_table = [
            self.prob[i] * np.product(self._pdf_normal_dist(i, x))
            for i in range(self.class_count)
        ]
        predicted = np.argmax(search_table)
        return self.classes[predicted]

    def _pdf_normal_dist(self, class_index, x):
        mean = self.mean[class_index]
        stdev = self.stdev[class_index]
        part1 = 1 / (stdev * np.sqrt(2 * np.pi))
        part2 = np.exp(-0.5 * ((x - mean) / stdev) ** 2)
        return part1 * part2


## Przeprowadzanie eksperymentów

### Funkcja wybierająca zbiór uczący

In [32]:
def get_training_and_validating_sets(data, chunk_size, chunk_to_skip):
    result = []
    skip_pos = chunk_to_skip * chunk_size
    for j in range(0, len(data), chunk_size):
        if skip_pos != j:
            result += data[j:j+chunk_size]
    return np.array(result), np.array(data[skip_pos:skip_pos+chunk_size])

### Pojedynczy eksperyment dla zadanych danych

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

def run_experiment(x, y, chunk_size, chunk_to_skip, test_size):
    x_test = x[:test_size]
    y_test = y[:test_size]
    x_train, x_valid = get_training_and_validating_sets(x[test_size:], chunk_size, chunk_to_skip)
    y_train, y_valid = get_training_and_validating_sets(y[test_size:], chunk_size, chunk_to_skip)
    algo = NaiveBayes(x_train, y_train)
    results = algo.classify(x_valid)

    print(f'Macierz pomyłek:\n{confusion_matrix(y_valid, results)}')
    print(f'Precyzja (dla konkretnych klas): {list(precision_score(y_valid, results, average=None))}')
    accuracy_valid = accuracy_score(y_valid, results)
    print(f'Precyzja: {100 * accuracy_valid:.2f}%')
    
    accuracy_test = accuracy_score(y_test, algo.classify(x_test))
    print(f'Precyzja dla zbioru testującego: {100 * accuracy_test:.2f}%')
    return accuracy_valid, accuracy_test

### Funkcja przeprowadzająca pełny eksperyment

Parametr `k` oznacza `k`-krotną walidację krzyżową.

In [34]:
from random import Random
from sklearn.datasets import load_iris

def experiment(k, seed, test_size=30):
    iris = load_iris()
    train_valid_size = len(iris.data) - test_size
    if train_valid_size % k != 0:
        print(f'Proszę wybrać `k` które dzieli liczbę {train_valid_size} bez reszty '
              f'({train_valid_size} % {k} = {train_valid_size % k})')
    else:
        zipped_iris = list(zip(iris.data, iris.target))
        Random(seed).shuffle(zipped_iris)
        iris.data, iris.target = zip(*zipped_iris)
        chunk_size = train_valid_size // k

        print(f'Rozmiar zbioru uczącego:      {(k - 1) * chunk_size}')
        print(f'Rozmiar zbioru walidacyjnego: {chunk_size}')
        print(f'Rozmiar zbioru testującego:   {test_size}')
        results_valid = []
        results_test = []
        for i in range(k):
            print('{:=^45}'.format(f' k = {i} '))
            accuracy_valid, accuracy_test = run_experiment(iris.data, iris.target, chunk_size, i, test_size)
            results_valid.append(accuracy_valid)
            results_test.append(accuracy_test)
        print('{:=^45}'.format(' PODSUMOWANIE '))
        print(f'Średnia precyzja trafień (zbiór walidacyjny): {100 * np.mean(results_valid):.2f}%')
        print(f'Średnia precyzja trafień (zbiór testujący):   {100 * np.mean(results_test):.2f}%')

## Eksperymenty dla zbioru testowego o rozmiarze 30

### Dla `k = 2`

In [35]:
SEED = 32145612469182
experiment(k=2, seed=SEED)

Rozmiar zbioru uczącego:      60
Rozmiar zbioru walidacyjnego: 60
Rozmiar zbioru testującego:   30
=================== k = 0 ===================
Macierz pomyłek:
[[22  0  0]
 [ 0 17  0]
 [ 0  1 20]]
Precyzja (dla konkretnych klas): [1.0, 0.9444444444444444, 1.0]
Precyzja: 98.33%
Precyzja dla zbioru testującego: 93.33%
=================== k = 1 ===================
Macierz pomyłek:
[[18  0  0]
 [ 0 23  3]
 [ 0  1 15]]
Precyzja (dla konkretnych klas): [1.0, 0.9583333333333334, 0.8333333333333334]
Precyzja: 93.33%
Precyzja dla zbioru testującego: 93.33%
=============== PODSUMOWANIE ================
Średnia precyzja trafień (zbiór walidacyjny): 95.83%
Średnia precyzja trafień (zbiór testujący):   93.33%


### Dla `k = 3`

In [36]:
experiment(k=3, seed=SEED)

Rozmiar zbioru uczącego:      80
Rozmiar zbioru walidacyjnego: 40
Rozmiar zbioru testującego:   30
=================== k = 0 ===================
Macierz pomyłek:
[[12  0  0]
 [ 0 10  0]
 [ 0  1 17]]
Precyzja (dla konkretnych klas): [1.0, 0.9090909090909091, 1.0]
Precyzja: 97.50%
Precyzja dla zbioru testującego: 93.33%
=================== k = 1 ===================
Macierz pomyłek:
[[15  0  0]
 [ 0 15  0]
 [ 0  1  9]]
Precyzja (dla konkretnych klas): [1.0, 0.9375, 1.0]
Precyzja: 97.50%
Precyzja dla zbioru testującego: 93.33%
=================== k = 2 ===================
Macierz pomyłek:
[[13  0  0]
 [ 0 17  1]
 [ 0  0  9]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.9]
Precyzja: 97.50%
Precyzja dla zbioru testującego: 93.33%
=============== PODSUMOWANIE ================
Średnia precyzja trafień (zbiór walidacyjny): 97.50%
Średnia precyzja trafień (zbiór testujący):   93.33%


### Dla `k = 5`

In [37]:
experiment(k=5, seed=SEED)

Rozmiar zbioru uczącego:      96
Rozmiar zbioru walidacyjnego: 24
Rozmiar zbioru testującego:   30
=================== k = 0 ===================
Macierz pomyłek:
[[ 7  0  0]
 [ 0  6  0]
 [ 0  0 11]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 1 ===================
Macierz pomyłek:
[[10  0  0]
 [ 0  6  0]
 [ 0  1  7]]
Precyzja (dla konkretnych klas): [1.0, 0.8571428571428571, 1.0]
Precyzja: 95.83%
Precyzja dla zbioru testującego: 93.33%
=================== k = 2 ===================
Macierz pomyłek:
[[ 8  0  0]
 [ 0 11  0]
 [ 0  1  4]]
Precyzja (dla konkretnych klas): [1.0, 0.9166666666666666, 1.0]
Precyzja: 95.83%
Precyzja dla zbioru testującego: 93.33%
=================== k = 3 ===================
Macierz pomyłek:
[[ 6  0  0]
 [ 0 11  1]
 [ 0  0  6]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.8571428571428571]
Precyzja: 95.83%
Precyzja dla zbioru testującego: 93.33%
=================== k = 4 =====

### Dla `k = 8`

In [38]:
experiment(k=8, seed=SEED)

Rozmiar zbioru uczącego:      105
Rozmiar zbioru walidacyjnego: 15
Rozmiar zbioru testującego:   30
=================== k = 0 ===================
Macierz pomyłek:
[[4 0 0]
 [0 3 0]
 [0 0 8]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 1 ===================
Macierz pomyłek:
[[4 0 0]
 [0 5 0]
 [0 1 5]]
Precyzja (dla konkretnych klas): [1.0, 0.8333333333333334, 1.0]
Precyzja: 93.33%
Precyzja dla zbioru testującego: 93.33%
=================== k = 2 ===================
Macierz pomyłek:
[[8 0 0]
 [0 2 0]
 [0 0 5]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 3 ===================
Macierz pomyłek:
[[6 0 0]
 [0 7 0]
 [0 0 2]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 4 ===================
Macierz pomyłek:
[[4 0 0]
 [0 7 0]
 [0 1 3]]
Pre

### Dla `k = 10`

In [39]:
experiment(k=10, seed=SEED)

Rozmiar zbioru uczącego:      108
Rozmiar zbioru walidacyjnego: 12
Rozmiar zbioru testującego:   30
=================== k = 0 ===================
Macierz pomyłek:
[[3 0 0]
 [0 2 0]
 [0 0 7]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 1 ===================
Macierz pomyłek:
[[4 0 0]
 [0 3 1]
 [0 0 4]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.8]
Precyzja: 91.67%
Precyzja dla zbioru testującego: 93.33%
=================== k = 2 ===================
Macierz pomyłek:
[[3 0 0]
 [0 4 0]
 [0 1 4]]
Precyzja (dla konkretnych klas): [1.0, 0.8, 1.0]
Precyzja: 91.67%
Precyzja dla zbioru testującego: 93.33%
=================== k = 3 ===================
Macierz pomyłek:
[[7 0 0]
 [0 2 0]
 [0 0 3]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 93.33%
=================== k = 4 ===================
Macierz pomyłek:
[[5 0 0]
 [0 5 0]
 [0 0 2]]
Precyzja (dla konkr

## Eksperymenty dla zbioru testowego o rozmiarze 10

### Dla k = 2

In [40]:
experiment(k=2, seed=SEED, test_size=10)

Rozmiar zbioru uczącego:      70
Rozmiar zbioru walidacyjnego: 70
Rozmiar zbioru testującego:   10
=================== k = 0 ===================
Macierz pomyłek:
[[24  0  0]
 [ 0 16  1]
 [ 0  2 27]]
Precyzja (dla konkretnych klas): [1.0, 0.8888888888888888, 0.9642857142857143]
Precyzja: 95.71%
Precyzja dla zbioru testującego: 100.00%
=================== k = 1 ===================
Macierz pomyłek:
[[22  0  0]
 [ 0 27  3]
 [ 0  0 18]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.8571428571428571]
Precyzja: 95.71%
Precyzja dla zbioru testującego: 100.00%
=============== PODSUMOWANIE ================
Średnia precyzja trafień (zbiór walidacyjny): 95.71%
Średnia precyzja trafień (zbiór testujący):   100.00%


### Dla k = 5

In [41]:
experiment(k=5, seed=SEED, test_size=10)

Rozmiar zbioru uczącego:      112
Rozmiar zbioru walidacyjnego: 28
Rozmiar zbioru testującego:   10
=================== k = 0 ===================
Macierz pomyłek:
[[ 6  0  0]
 [ 0  5  1]
 [ 0  1 15]]
Precyzja (dla konkretnych klas): [1.0, 0.8333333333333334, 0.9375]
Precyzja: 92.86%
Precyzja dla zbioru testującego: 100.00%
=================== k = 1 ===================
Macierz pomyłek:
[[10  0  0]
 [ 0  7  1]
 [ 0  1  9]]
Precyzja (dla konkretnych klas): [1.0, 0.875, 0.9]
Precyzja: 92.86%
Precyzja dla zbioru testującego: 100.00%
=================== k = 2 ===================
Macierz pomyłek:
[[12  0  0]
 [ 0  9  0]
 [ 0  1  6]]
Precyzja (dla konkretnych klas): [1.0, 0.9, 1.0]
Precyzja: 96.43%
Precyzja dla zbioru testującego: 100.00%
=================== k = 3 ===================
Macierz pomyłek:
[[ 8  0  0]
 [ 0 13  1]
 [ 0  0  6]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.8571428571428571]
Precyzja: 96.43%
Precyzja dla zbioru testującego: 100.00%
=================== k = 4 ===========

### Dla k = 7

In [42]:
experiment(k=7, seed=SEED, test_size=10)

Rozmiar zbioru uczącego:      120
Rozmiar zbioru walidacyjnego: 20
Rozmiar zbioru testującego:   10
=================== k = 0 ===================
Macierz pomyłek:
[[6 0 0]
 [0 3 1]
 [0 1 9]]
Precyzja (dla konkretnych klas): [1.0, 0.75, 0.9]
Precyzja: 90.00%
Precyzja dla zbioru testującego: 100.00%
=================== k = 1 ===================
Macierz pomyłek:
[[6 0 0]
 [0 4 1]
 [0 0 9]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.9]
Precyzja: 95.00%
Precyzja dla zbioru testującego: 100.00%
=================== k = 2 ===================
Macierz pomyłek:
[[6 0 0]
 [0 5 0]
 [0 1 8]]
Precyzja (dla konkretnych klas): [1.0, 0.8333333333333334, 1.0]
Precyzja: 95.00%
Precyzja dla zbioru testującego: 100.00%
=================== k = 3 ===================
Macierz pomyłek:
[[10  0  0]
 [ 0  7  0]
 [ 0  0  3]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 100.00%
=================== k = 4 ===================
Macierz pomyłek:
[[5 0 0]
 [0 8 0]
 

## Eksperymenty dla zbioru testowego o rozmiarze 50

### Dla `k = 2`

In [43]:
experiment(k=2, seed=SEED, test_size=50)

Rozmiar zbioru uczącego:      50
Rozmiar zbioru walidacyjnego: 50
Rozmiar zbioru testującego:   50
=================== k = 0 ===================
Macierz pomyłek:
[[19  0  0]
 [ 0 16  0]
 [ 0  2 13]]
Precyzja (dla konkretnych klas): [1.0, 0.8888888888888888, 1.0]
Precyzja: 96.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 1 ===================
Macierz pomyłek:
[[15  0  0]
 [ 0 21  1]
 [ 0  0 13]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.9285714285714286]
Precyzja: 98.00%
Precyzja dla zbioru testującego: 94.00%
=============== PODSUMOWANIE ================
Średnia precyzja trafień (zbiór walidacyjny): 97.00%
Średnia precyzja trafień (zbiór testujący):   95.00%


### Dla `k = 5`

In [44]:
experiment(k=5, seed=SEED, test_size=50)

Rozmiar zbioru uczącego:      80
Rozmiar zbioru walidacyjnego: 20
Rozmiar zbioru testującego:   50
=================== k = 0 ===================
Macierz pomyłek:
[[6 0 0]
 [0 5 0]
 [0 1 8]]
Precyzja (dla konkretnych klas): [1.0, 0.8333333333333334, 1.0]
Precyzja: 95.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 1 ===================
Macierz pomyłek:
[[10  0  0]
 [ 0  7  0]
 [ 0  0  3]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 94.00%
=================== k = 2 ===================
Macierz pomyłek:
[[5 0 0]
 [0 8 0]
 [0 1 6]]
Precyzja (dla konkretnych klas): [1.0, 0.8888888888888888, 1.0]
Precyzja: 95.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 3 ===================
Macierz pomyłek:
[[7 0 0]
 [0 8 2]
 [0 0 3]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 0.6]
Precyzja: 90.00%
Precyzja dla zbioru testującego: 94.00%
=================== k = 4 ===================
Macierz pomyłek:
[[6 0 0]
 

### Dla `k = 10`

In [45]:
experiment(k=10, seed=SEED, test_size=50)

Rozmiar zbioru uczącego:      90
Rozmiar zbioru walidacyjnego: 10
Rozmiar zbioru testującego:   50
=================== k = 0 ===================
Macierz pomyłek:
[[2 0 0]
 [0 3 0]
 [0 1 4]]
Precyzja (dla konkretnych klas): [1.0, 0.75, 1.0]
Precyzja: 90.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 1 ===================
Macierz pomyłek:
[[4 0 0]
 [0 2 0]
 [0 0 4]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 2 ===================
Macierz pomyłek:
[[6 0 0]
 [0 3 0]
 [0 0 1]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 94.00%
=================== k = 3 ===================
Macierz pomyłek:
[[4 0 0]
 [0 4 0]
 [0 0 2]]
Precyzja (dla konkretnych klas): [1.0, 1.0, 1.0]
Precyzja: 100.00%
Precyzja dla zbioru testującego: 96.00%
=================== k = 4 ===================
Macierz pomyłek:
[[3 0 0]
 [0 4 0]
 [0 1 2]]
Precyzja (dla konk

## Podsumowanie i wnioski